<a href="https://colab.research.google.com/github/MITHRAMA/Resume_Screening_Using_DeepLearning/blob/main/Resume_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd

In [51]:
df = pd.read_csv('/content/job_descriptions.csv')
df.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."


In [52]:
null_counts = df.isnull().sum()
print("Null value counts per column:\n", null_counts)
df['skills'] = df['skills'].str.replace('[^\w\s]','')

Null value counts per column:
 Job Id                0
Experience            0
Qualifications        0
Salary Range          0
location              0
Country               0
latitude              0
longitude             0
Work Type             0
Company Size          0
Job Posting Date      0
Preference            0
Contact Person        0
Contact               0
Job Title             0
Role                  0
Job Portal            0
Job Description       1
Benefits              1
skills                1
Responsibilities      1
Company               1
Company Profile     803
dtype: int64


In [53]:
num_unique_companies = df['Company'].nunique()
print("Number of unique companies:", num_unique_companies)


Number of unique companies: 888


In [55]:
!pip install PyPDF2



In [56]:
from google.colab import files
from ipywidgets import Dropdown, FileUpload, Button, Output
from PyPDF2 import PdfReader
import io

In [57]:

global predicted_completeness_status_percentage #overall
global skills_match_percentage #skills
global length_percentage #length
global average_percentage #Completness
global total_percentage_contact_info #contact_info

# Function to filter job titles based on selected company
def get_job_titles(company):
    return df[df['Company'] == company]['Job Title'].unique()

# Define widgets
company_dropdown = Dropdown(options=df['Company'].unique(), description='Company:')
job_title_dropdown = Dropdown(description='Job Title:')
upload_resume = FileUpload(description='Upload Resume', accept='.pdf')
submit_button = Button(description='Submit')
output = Output()


# Function to handle button click
def on_submit_button_click(b):
    global r_skills, selected_company, selected_job_title, resume_text
    selected_company = company_dropdown.value
    selected_job_title = job_title_dropdown.value
    resume_pdf = next(iter(upload_resume.value.values()))['content']

    # Convert PDF content to text
    resume_text = extract_text_from_pdf(resume_pdf)

    # Display the selected information
    with output:
        output.clear_output()
        print("\n")
        print("\n")
        print("\n")
        print("Resume Text:")
        print(resume_text)
        print("\n")
        print("\n")
        print("\n")
        print("Selected Company:", selected_company)
        print("Selected Job Title:", selected_job_title)
        r_skills = df[(df['Company'] == selected_company) & (df['Job Title'] == selected_job_title)]['skills'].values[0]
        print("Skills for Selected Job Title:", r_skills)


# Update job titles dropdown options when company changes
def on_company_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        job_titles = get_job_titles(change['new'])
        job_title_dropdown.options = job_titles


# Function to extract text from PDF
def extract_text_from_pdf(pdf_content):
    pdf_reader = PdfReader(io.BytesIO(pdf_content))
    text = ''
    for page_num in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()
    return text

# Link dropdowns with change handlers
company_dropdown.observe(on_company_change)

# Link button with click handler
submit_button.on_click(on_submit_button_click)

# Display the widgets
display(company_dropdown, job_title_dropdown, upload_resume, submit_button, output)


Dropdown(description='Company:', options=('Icahn Enterprises', 'PNC Financial Services Group', 'United Service…

Dropdown(description='Job Title:', options=(), value=None)

FileUpload(value={}, accept='.pdf', description='Upload Resume')

Button(description='Submit', style=ButtonStyle())

Output()

In [58]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import string
nltk.download('punkt')



# Define a set of punctuation characters
punctuation = set(string.punctuation)

# Tokenize the skills text using NLTK and filter out special characters
tokens = [word for word in word_tokenize(r_skills) if word.isalnum()]


# Count the frequency of each word
word_freq = Counter(tokens)

# Store the word frequency list in a variable
word_freq_variable = word_freq.items()

# Display the variable
print("Variable containing word frequency list:", word_freq_variable)


Variable containing word frequency list: dict_items([('Java', 2), ('programming', 1), ('frameworks', 1), ('Spring', 1), ('design', 1), ('Code', 1), ('debugging', 1), ('Software', 1), ('development', 1), ('skills', 1)])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Tokenize the resume text
resume_tokens = word_tokenize(resume_text)

# Calculate the frequency of each token
token_freq = Counter(resume_tokens)

# Store the token frequency list in a variable
token_freq_variable = token_freq.items()

# Display the variable
print("Variable containing token frequency list:", token_freq_variable)


Variable containing token frequency list: dict_items([('V', 2), ('G', 1), ('P', 1), ('MITHRAMA', 1), ('MERN', 4), ('Stack', 3), ('Enthusiast', 1), ('and', 10), ('Window-based', 2), ('Applications', 1), ('Developer', 3), ('♂¶obile+91', 1), ('7981085957', 1), ('mithramavgvp', 1), ('@', 1), ('gmail.com', 1), ('/h⌢meIndia', 1), (',', 39), ('Hyderabad', 2), ('/linkedinlinkedIn.com', 1), ('/githubgithub.com', 1), ('Xx.com', 1), ('Skills', 3), ('•Languages', 1), (':', 4), ('Java', 7), ('C++', 2), ('C', 3), ('Python', 2), ('Javascript', 1), ('HTML', 1), ('CSS', 1), ('PHP', 1), ('#', 1), ('.net', 1), ('•Database', 1), ('MongoDB', 2), ('MySQL', 1), ('•Technologies/Technical', 1), ('Data', 2), ('Structures', 1), ('Algorithms', 1), ('stack', 2), ('Swing', 2), ('full', 1), ('development', 2), ('using', 3), ('JAVA', 1), ('Operating', 1), ('Systems', 2), ('DBMS', 2), ('Computer', 3), ('Hardware', 1), ('Embedded', 1), ('8051MC', 1), ('Machine', 1), ('Learning', 2), ('Deep', 1), ('Microservices', 1), (

In [60]:
# Convert the dictionaries to sets of lowercase keys
word_keys = set(word.lower() for word in word_freq.keys())
token_keys = set(token.lower() for token in token_freq.keys())

# Calculate the intersection of lowercase keys (common tokens)
common_keys = word_keys.intersection(token_keys)

# Calculate the match percentage
if word_keys == token_keys:
    match_percentage = 100.0
else:
    match_percentage = (len(common_keys) / len(word_keys)) * 100.0

# Print the match percentage
print("Match Percentage:", match_percentage)
print("Common Keys:", common_keys)
print(len(word_keys))

Match Percentage: 50.0
Common Keys: {'programming', 'java', 'skills', 'spring', 'development'}
10


In [61]:
# Calculate the total frequency
total_frequency = sum(freq for _, freq in token_freq_variable)

# Determine the match percentage
if 300 <= total_frequency <= 800:
    length_percentage = 100
else:
    length_percentage = 50

# Print the match percentage
print("length Percentage:", length_percentage, "%")


length Percentage: 100 %


In [64]:

import tensorflow as tf
from tensorflow.keras import layers, models

# Example resume text (replace with your actual resume text)
#resume_text = "I am eating cake."

# Convert resume text to numerical representation (example: using word embeddings)
# Replace this step with your actual preprocessing method
# For simplicity, let's assume each word in the resume text is represented by a unique integer
word_to_index = {word: i for i, word in enumerate(resume_text.split())}
numerical_representation = [word_to_index[word] for word in resume_text.split()]

# Pad or truncate the numerical representation to a fixed length (example: 100)
max_sequence_length = 100
padded_representation = tf.keras.preprocessing.sequence.pad_sequences([numerical_representation], maxlen=max_sequence_length)

# Define RNN model
model = models.Sequential([
    layers.Embedding(input_dim=len(word_to_index) + 1, output_dim=64, input_length=max_sequence_length),#converts nr into
    #dense vectors of fixed size
    layers.LSTM(64),#process seq data
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Convert padded_representation to float32 data type
padded_representation = padded_representation.astype(np.float32)

# Predict completeness status
predicted_completeness_status = model.predict(padded_representation)

# Print predicted completeness status
print("Predicted Completeness Status:", predicted_completeness_status)







1/1 [==============================] - 0s 452ms/step
Predicted Completeness Status: [[0.50354564]]


In [65]:

# Print rounded completeness status
pcs_percentage= predicted_completeness_status*100
print("Rounded Predicted Completeness Status (multiplied by 2):", pcs_percentage )


Rounded Predicted Completeness Status (multiplied by 2): [[50.354565]]


In [70]:
import spacy
import re

# Load the English NER model from spaCy
nlp = spacy.load("en_core_web_sm")

# Define regular expressions for email addresses and phone numbers
email_regex = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
phone_regex = r"\b(?:\+\d{1,2}\s?)?(?:\d{10})\b"
'''
# Sample resume text
resume_text = """
V G V P MITHRAMA MERN Stack Enthusiast and Window-based Applications Developer ♂¶obile+91 7981085957 mithramavgvp@gmail.com /h⌢meIndia, Hyderabad /linkedinlinkedIn.com /githubgithub.com Xx.com Skills •Languages: Java, C++, C, Python, Javascript, HTML, CSS, PHP, C#.net •Database: MongoDB and MySQL •Technologies/Technical Skills: Data Structures and Algorithms, MERN stack, Java Swing, full stack development using JAVA, Operating Systems, DBMS, Computer Hardware, Embedded Systems 8051MC, Machine Learning, Deep Learning, Microservices with Spring Boot, Ethernet cable configuration and color coding •Soft Skills: Presentation, Organized, Creative Problem-Solving, Teamwork, Active Listening, Analytical Thinking Education BE Information Technology Chaitanya Bharathi Institute of Technology Gandipet, Hyderabad 2022-2025 CGPA:9.12 Diploma Computer Science Government Polytechnic for Women Secunderabad 2019-2022 CGPA:9.93 Projects Medi Management System, Window-based Application •Enhanced medicine inventory management through a Java Swing GUI, streamlining operations. •Implemented real-time data updates and automated alerts, optimizing stock management and customer service College Mentoring System using the MERN Stack •Led MERN Stack-based ”College Mentoring System” development under Prof. K. Gangadhar Rao. •Implemented a pivotal feature for fetching student marks and presenting performance analytics, enhancing the efficiency of mentorship interactions. Find Best Laptop Website using Java Full Stack •Developed a user-centric website for personalized laptop recommendations and Implemented interface for budget constraints Internships & Experiences Programming Languages Mentor Startup
"""
'''
# Process the resume text with spaCy
doc = nlp(resume_text)

# Initialize variables to count the presence of each entity
has_name = False
has_phone_number = False
has_email = False
has_place = False

# Iterate over the entities in the document
for ent in doc.ents:
    if ent.label_ == "PERSON":  # Check if the entity is a person name
        has_name = True
    elif ent.label_ == "GPE":  # Check if the entity is a geopolitical entity (e.g., place)
        has_place = True

# Check for email addresses using regular expression
if re.search(email_regex, resume_text):
    has_email = True

# Check for phone numbers using regular expression
if re.search(phone_regex, resume_text):
    has_phone_number = True

# Calculate percentages for each entity
name_percentage = 25 if has_name else 0
phone_number_percentage = 25 if has_phone_number else 0
email_percentage = 25 if has_email else 0
place_percentage = 25 if has_place else 0

# Calculate the total completeness percentage
total_percentage_contact_info = name_percentage + phone_number_percentage + email_percentage + place_percentage

# Print the completeness percentages
print("Name Percentage:", name_percentage)
print("Phone Number Percentage:", phone_number_percentage)
print("Email Percentage:", email_percentage)
print("Place Percentage:", place_percentage)
print("Total Percentage for 100:", total_percentage_contact_info)


Name Percentage: 25
Phone Number Percentage: 25
Email Percentage: 25
Place Percentage: 25
Total Percentage for 100: 100


In [67]:
average_percentage = (pcs_percentage + match_percentage + length_percentage+total_percentage_contact_info) / 4
average_percentage

array([[50.088642]], dtype=float32)

In [68]:
from IPython.display import display, HTML

# Assuming pcs_percentage, match_percentage, length_percentage, and average_percentage are NumPy arrays
# Extract scalar values from the arrays
p = pcs_percentage.item()
s = match_percentage  # Extracting scalar value from numpy array
l = length_percentage  # Extracting scalar value from numpy array
i = total_percentage_contact_info
a = average_percentage.item()  # Extracting scalar value from numpy array

# Prepare HTML content for the web page
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Resume Analysis</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #f0f0f0;
            margin: 0;
            padding: 0;
        }}
        .container {{
            max-width: 800px;
            margin: 50px auto;
            background-color: #fff;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }}
        h1 {{
            font-size: 24px;
            color: #333;
        }}
        p {{
            font-size: 18px;
            color: #666;
        }}
        .big-text {{
            font-size: 36px;
            font-weight: bold;
        }}
        .highlight {{
            background-color: #ffd700;
            padding: 5px;
            border-radius: 4px;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1>Resume Analysis</h1>
        <p>Predicted Completeness Status Percentage: <span class="highlight">{:.0f}%</span></p>
        <p>Skills Match Percentage: <span class="highlight">{:.0f}%</span></p>
        <p>Length Percentage: <span class="highlight">{:.0f}%</span></p>
        <p>Clear Contact info: <span class="highlight">{:.0f}%</span></p>
        <h2>Average Percentage for 100%</h2>
        <p class="big-text"><span class="highlight">{:.0f}%</span></p>
    </div>
</body>
</html>""".format(p, s, l, i, a)

# Display the HTML content
display(HTML(html_content))
